In [1]:
%config IPCompleter.greedy=True


from sklearn.cluster import KMeans, DBSCAN, MeanShift
from sklearn import datasets
from sklearn.manifold import TSNE
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
from pylab import plot, show
from sklearn.decomposition import PCA
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import silhouette_samples, silhouette_score
import pandas as pd
import numpy as np


In [2]:

arr = np.genfromtxt('../data/text_vectors_lane_hidden_state_200.csv', delimiter=',')
arr
# df = pd.read_csv('[Query-List of sequences]-MT-human_clean40_genome_embeddings_sigmoid.csv')

array([[-2.28603967e-05, -1.06819032e-03,  7.24777274e-05, ...,
         4.83475000e-01, -1.49935513e-04, -1.02330244e-03],
       [ 1.94452573e-02, -1.96926226e-03,  7.75452331e-03, ...,
        -6.87566102e-02, -3.06257993e-01,  3.22896689e-01],
       [ 1.04616001e-01, -6.39605522e-02,  6.90813288e-02, ...,
        -3.07952607e-04, -3.71909291e-01,  3.14261802e-02],
       ...,
       [-4.81366320e-03, -2.08850224e-02,  5.67212813e-02, ...,
         1.53312099e-03, -6.15073740e-02,  4.05471474e-02],
       [-8.34628195e-02, -2.48330403e-02,  6.09705634e-02, ...,
        -1.50890589e-01, -1.30891681e-01,  1.00428768e-01],
       [-2.08488875e-03, -3.02624051e-03,  1.29533233e-02, ...,
        -5.53351315e-03,  7.92861342e-01,  9.13293660e-03]])

In [3]:
# df

In [4]:
# arr = df.values

In [5]:
arr

array([[-2.28603967e-05, -1.06819032e-03,  7.24777274e-05, ...,
         4.83475000e-01, -1.49935513e-04, -1.02330244e-03],
       [ 1.94452573e-02, -1.96926226e-03,  7.75452331e-03, ...,
        -6.87566102e-02, -3.06257993e-01,  3.22896689e-01],
       [ 1.04616001e-01, -6.39605522e-02,  6.90813288e-02, ...,
        -3.07952607e-04, -3.71909291e-01,  3.14261802e-02],
       ...,
       [-4.81366320e-03, -2.08850224e-02,  5.67212813e-02, ...,
         1.53312099e-03, -6.15073740e-02,  4.05471474e-02],
       [-8.34628195e-02, -2.48330403e-02,  6.09705634e-02, ...,
        -1.50890589e-01, -1.30891681e-01,  1.00428768e-01],
       [-2.08488875e-03, -3.02624051e-03,  1.29533233e-02, ...,
        -5.53351315e-03,  7.92861342e-01,  9.13293660e-03]])

In [6]:
len(arr)

140

In [7]:
len(arr[0])

200

In [8]:
def evaluate_kmeans():
    max_score = -1
    predicted_labels_final = []
    clusters = 0
    for i in range(2, 100):
        model = KMeans(n_clusters=i)
        model.fit(arr)
        predicted_labels = model.predict(arr)
        silhouette_avg = silhouette_score(arr, predicted_labels)
        
#         print (i)
#         print (silhouette_avg)
        if(max_score < silhouette_avg):
            max_score = silhouette_avg
            predicted_labels_final = predicted_labels
            clusters = i
    return max_score, predicted_labels_final, clusters

In [9]:
def evaluate_gmm():
    max_score = -1
    predicted_labels_final = []
    clusters = 0
    for i in range(2, 100):
        gmm = GaussianMixture(n_components= i, init_params='kmeans')
        gmm = gmm.fit(arr)
        predicted_labels = gmm.predict(arr)
        silhouette_avg = silhouette_score(arr, predicted_labels)
        if(max_score < silhouette_avg):
            max_score = silhouette_avg
            predicted_labels_final = predicted_labels
            clusters = i
            
    for i in range(2, 100):
        gmm = GaussianMixture(n_components= i, init_params='random')
        gmm = gmm.fit(arr)
        predicted_labels = gmm.predict(arr)
        silhouette_avg = silhouette_score(arr, predicted_labels)
        if(max_score < silhouette_avg):
            max_score = silhouette_avg
            predicted_labels_final = predicted_labels
            clusters = i
            
    return max_score, predicted_labels_final, clusters

In [10]:
def evaluate_dbscan():
    max_score = -1
    predicted_labels_final = []
    clusters = 0
    for i in range(1, 10000, 1):
        for j in range(50, 200, 10):
            eps = i*0.5
            dbscan = DBSCAN(eps=i,min_samples=j)
            dbscan.fit(arr)
            predicted_labels = dbscan.labels_
#             print (eps)
#             print (j)
#             print (predicted_labels)
            
            labels, count = np.unique(predicted_labels, return_counts= True)
            if (len(labels) == 1):
                continue
            silhouette_avg = silhouette_score(arr, predicted_labels)
            silhouette_avg
            if(max_score < silhouette_avg):
                max_score = silhouette_avg
                predicted_labels_final = predicted_labels
                clusters = labels
                
    return max_score, predicted_labels_final, clusters

In [11]:
score_kmeans, predicted_label_kmeans, cluster_kmeans = evaluate_kmeans()
score_kmeans, predicted_label_kmeans, cluster_kmeans

/Users/richa/opt/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (90) found smaller than n_clusters (91). Possibly due to duplicate points in X.
  return_n_iter=True)
/Users/richa/opt/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (90) found smaller than n_clusters (92). Possibly due to duplicate points in X.
  return_n_iter=True)
/Users/richa/opt/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (90) found smaller than n_clusters (93). Possibly due to duplicate points in X.
  return_n_iter=True)
/Users/richa/opt/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (90) found smaller than n_clusters (94). Possibly due to duplicate points in X.
  return_n_iter=True)
/Users/richa/opt/anaconda3/lib/python3.7/site-pa

(0.7142857026411704,
 array([12, 13, 14, 33, 15, 51,  6, 18, 23, 44, 35, 25, 50, 75, 11, 20,  4,
        42,  1, 58, 56, 27, 34, 10,  9,  0, 22, 32, 41, 57, 54,  8, 59, 55,
        68, 26, 69,  3, 37, 17, 48, 45, 16,  2,  7,  5, 52, 38, 76, 70, 12,
        13, 14, 33, 15, 51,  6, 18, 23, 44, 35, 25, 50, 75, 11, 20,  4, 42,
         1, 58, 56, 27, 34, 10,  9,  0, 22, 32, 41, 57, 54,  8, 59, 55, 68,
        26, 69,  3, 37, 17, 48, 45, 16,  2,  7,  5, 52, 38, 76, 70, 86, 82,
        87, 73, 64, 66, 40, 28, 36, 77, 49, 24, 61, 43, 89, 74, 71, 88, 83,
        29, 62, 67, 46, 63, 60, 79, 47, 85, 81, 72, 80, 30, 39, 53, 31, 65,
        19, 78, 84, 21], dtype=int32),
 90)

In [12]:
score_kmeans, predicted_label_kmeans, cluster_kmeans

(0.7142857026411704,
 array([12, 13, 14, 33, 15, 51,  6, 18, 23, 44, 35, 25, 50, 75, 11, 20,  4,
        42,  1, 58, 56, 27, 34, 10,  9,  0, 22, 32, 41, 57, 54,  8, 59, 55,
        68, 26, 69,  3, 37, 17, 48, 45, 16,  2,  7,  5, 52, 38, 76, 70, 12,
        13, 14, 33, 15, 51,  6, 18, 23, 44, 35, 25, 50, 75, 11, 20,  4, 42,
         1, 58, 56, 27, 34, 10,  9,  0, 22, 32, 41, 57, 54,  8, 59, 55, 68,
        26, 69,  3, 37, 17, 48, 45, 16,  2,  7,  5, 52, 38, 76, 70, 86, 82,
        87, 73, 64, 66, 40, 28, 36, 77, 49, 24, 61, 43, 89, 74, 71, 88, 83,
        29, 62, 67, 46, 63, 60, 79, 47, 85, 81, 72, 80, 30, 39, 53, 31, 65,
        19, 78, 84, 21], dtype=int32),
 90)

In [13]:
score_gmm, predicted_label_gmm, cluster_gmm = evaluate_gmm()

/Users/richa/opt/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (90) found smaller than n_clusters (91). Possibly due to duplicate points in X.
  return_n_iter=True)
/Users/richa/opt/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (90) found smaller than n_clusters (92). Possibly due to duplicate points in X.
  return_n_iter=True)
/Users/richa/opt/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (90) found smaller than n_clusters (93). Possibly due to duplicate points in X.
  return_n_iter=True)
/Users/richa/opt/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (90) found smaller than n_clusters (94). Possibly due to duplicate points in X.
  return_n_iter=True)
/Users/richa/opt/anaconda3/lib/python3.7/site-pa

In [14]:
score_dbscan, predicted_label_dbscan, clusters_dbscan = evaluate_dbscan()

In [15]:
 print ('KMeans results: ')
print ('Silhoeutte Score :', score_kmeans)
print ('Number of clusters formed: ', cluster_kmeans)

KMeans results: 
Silhoeutte Score : 0.7142857026411704
Number of clusters formed:  90


###### predicted_labels_Gmm

In [16]:
print ('GMM results: ')
print ('Silhoeutte Score :', score_gmm)
print ('Number of clusters formed: ', cluster_gmm)

GMM results: 
Silhoeutte Score : 0.7142857026411704
Number of clusters formed:  90


In [17]:
score_dbscan, predicted_label_dbscan
print ('DBSCAN results: ')
print ('Silhoeutte Score :', score_dbscan)
print ('Number of clusters formed: ', clusters_dbscan)

DBSCAN results: 
Silhoeutte Score : 0.2821206953089342
Number of clusters formed:  [-1  0]


In [18]:
predicted_label_dbscan

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0, -1, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       -1, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       -1,  0,  0, -1])